In [2]:
#import necessary packages
import pandas as pd
import requests
import lxml
import html5lib
from bs4 import BeautifulSoup
import time # To put the system to sleep
import random # for random numbers
import warnings
import sqlite3
import missingno as miss
from selenium import webdriver
warnings.filterwarnings('ignore')


In [11]:
#urls for emotional health

#2017-2019
url_depression_19 = "https://www.kidsdata.org/topic/662/depressive-feelings-grade/table#fmt=943&loc=2,127,347,1763,331,348,336,171,321,345,357,332,324,369,358,362,360,337,327,364,356,217,353,328,354,323,352,320,339,334,365,343,330,367,344,355,366,368,265,349,361,4,273,59,370,326,333,322,341,338,350,342,329,325,359,351,363,340,335&tf=134&ch=69,305,306,431,1177,1176&sortColumnId=0&sortType=asc"
#2015 - 2017
url_depression_17 = "https://www.kidsdata.org/topic/662/depressive-feelings-grade/table#fmt=943&loc=2,127,347,1763,331,348,336,171,321,345,357,332,324,369,358,362,360,337,327,364,356,217,353,328,354,323,352,320,339,334,365,343,330,367,344,355,366,368,265,349,361,4,273,59,370,326,333,322,341,338,350,342,329,325,359,351,363,340,335&tf=122&ch=69,305,306,431,1177,1176&sortColumnId=0&sortType=asc"
#2013 - 2015
url_depression_15 = "https://www.kidsdata.org/topic/662/depressive-feelings-grade/table#fmt=943&loc=2,127,347,1763,331,348,336,171,321,345,357,332,324,369,358,362,360,337,327,364,356,217,353,328,354,323,352,320,339,334,365,343,330,367,344,355,366,368,265,349,361,4,273,59,370,326,333,322,341,338,350,342,329,325,359,351,363,340,335&tf=93&ch=69,305,306,431,1177,1176&sortColumnId=0&sortType=asc"
#2011-2013
url_depression_13 = "https://www.kidsdata.org/topic/662/depressive-feelings-grade/table#fmt=943&loc=2,127,347,1763,331,348,336,171,321,345,357,332,324,369,358,362,360,337,327,364,356,217,353,328,354,323,352,320,339,334,365,343,330,367,344,355,366,368,265,349,361,4,273,59,370,326,333,322,341,338,350,342,329,325,359,351,363,340,335&tf=81&ch=69,305,306,431,1177,1176&sortColumnId=0&sortType=asc"
urls_depression = [url_depression_19, url_depression_17, url_depression_15, url_depression_13]

driver = webdriver.Chrome('C:/chromedriver.exe')
driver.get(url_depression_19)
html = driver.page_source

tables = pd.read_html(html)


In [30]:

#download the data
depression_13 = pd.read_csv("depression_13.csv")
depression_13["year"] = "2013"

depression_15 = pd.read_csv("depression_15.csv")
depression_15["year"] = "2015"

depression_17 = pd.read_csv("depression_17.csv")
depression_17["year"] = "2017"

depression_all = pd.concat([depression_13, depression_15, depression_17])

depression_all = depression_all[depression_all['Data'] != "N_A"]
depression_all = depression_all[depression_all['Data'] != "S"]
depression_all = depression_all[depression_all["StudentResponse"] != "No"]

depression_all = depression_all.drop(columns= ["FIPSCode", "LocationType", "DataFormat", "TimeFrame", "StudentResponse"])

depression_all = depression_all.rename(columns={"Data": "students_reporting_feelings_of_depression"})
depression_all["students_reporting_feelings_of_depression"] = depression_all["students_reporting_feelings_of_depression"].astype(float)
depression_all = depression_all.reset_index(drop=True)

depression_all = depression_all.sort_values(by=["Location", "year", "GradeLevel"])

da_pre_dummy = depression_all

#dummy grade level
grade_dummies = pd.get_dummies(depression_all.GradeLevel)
grade_dummies.columns = [c.lower().replace(" ","_") for c in grade_dummies.columns]
grade_dummies = grade_dummies.drop(['non_traditional'],axis=1) # Baseline
depression_all = pd.concat([depression_all.drop(['GradeLevel'],axis=1),grade_dummies],axis=1)

#country grade level
c_dummies = pd.get_dummies(depression_all.Location)
c_dummies.columns = [c.lower().replace(" ","_") for c in c_dummies.columns]
c_dummies = c_dummies.drop(['alameda_county'],axis=1) # Baseline
depression_all = pd.concat([depression_all.drop(['Location'],axis=1),c_dummies],axis=1)



#cleaning data for bullying
bullying_13 = pd.read_csv("bullying_13.csv")
bullying_13["year"] = "2013"

bullying_15 = pd.read_csv("bullying_15.csv")
bullying_15["year"] = "2015"

bullying_17 = pd.read_csv("bullying_17.csv")
bullying_17["year"] = "2017"

bullying_all = pd.concat([bullying_13, bullying_15, bullying_17])
bullying_all = bullying_all.drop(columns= ["FIPSCode", "LocationType", "DataFormat", "TimeFrame", ])

bullying_all = bullying_all[bullying_all['Data'] != "N_A"]
bullying_all = bullying_all[bullying_all['Data'] != "S"]
bullying_all = bullying_all[bullying_all['StudentResponse'] != "None"]
bullying_all = bullying_all.drop(columns=["StudentResponse"])
bullying_all = bullying_all.rename(columns={"Data": "students_bullied_at_school"})

bullying_all = bullying_all.sort_values(by=["Location", "year", "GradeLevel"])
bullying_all = bullying_all.reset_index(drop=True)


#cleaning data for gang involvement
gang_13 = pd.read_csv("gang_13.csv")
gang_13["year"] = "2013"

gang_15 = pd.read_csv("gang_15.csv")
gang_15["year"] = "2015"

gang_17 = pd.read_csv("gang_17.csv")
gang_17["year"] = "2017"

gang_all = pd.concat([gang_13, gang_15, gang_17])

gang_all = gang_all[gang_all['Data'] != "N_A"]
gang_all = gang_all[gang_all['Data'] != "S"]
gang_all = gang_all[gang_all['StudentResponse'] != "No"]

gang_all = gang_all.drop(columns= ["FIPSCode", "LocationType", "DataFormat", "TimeFrame", "StudentResponse"])
gang_all = gang_all.rename(columns={"Data": "students_involved_with_gang_activity"})

gang_all = gang_all.sort_values(by=["Location", "year", "GradeLevel"])
gang_all = gang_all[["Location", "year", "GradeLevel", "students_involved_with_gang_activity"]]
gang_all = gang_all.reset_index(drop=True)


#data for truancy
tr_7 = pd.read_csv("truancy_all.csv")
tr_7["GradeLevel"] = "Grade 7"

tr_9 = pd.read_csv("truancy_all.csv")
tr_9["GradeLevel"] = "Grade 9"

tr_11 = pd.read_csv("truancy_all.csv")
tr_11["GradeLevel"] = "Grade 11"

tr_nt = pd.read_csv("truancy_all.csv") 
tr_nt["GradeLevel"] = "Non_traditional"

tr_all = pd.concat([tr_7, tr_9, tr_11, tr_nt])
tr_all = tr_all.sort_values(by=["Location"])
tr_all = tr_all.replace("2016", "2017")

tr_all = tr_all.drop(columns=["FIPSCode", "LocationType", "DataFormat"])

tr_all = tr_all[tr_all['Data'] != "N_A"]
tr_all = tr_all[tr_all['Data'] != "S"]

tr_all = tr_all.rename(columns={"Data": "students_truant_yearly_per_100"})
tr_all = tr_all.rename(columns={"TimeFrame" : "year"})
tr_all["year"] = tr_all["year"].astype(object)
tr_all = tr_all.sort_values(by=["Location", "year", "GradeLevel"])
tr_all = tr_all.reset_index(drop=True)


#create feature dataframe
#bullying all is the shortest so we left join to that
df_factors = gang_all.merge(bullying_all, how="left", on =["Location", "GradeLevel", "year"] )
df_factors_tr = df_factors.merge(tr_all, how='left', on =["Location"] )
tr = df_factors_tr["students_truant_yearly_per_100"].astype("float")

df_factors["students_truant_yearly_per_100"] = tr
df_factors["students_involved_with_gang_activity"] = df_factors["students_involved_with_gang_activity"].astype("float")
df_factors["students_bullied_at_school"] = df_factors["students_bullied_at_school"].astype("float")